# <font color='#6629b2'>Predicting sentiment ratings with RNN using Keras</font>


## <font color='#6629b2'>Study Material</font>
- https://web.stanford.edu/~jurafsky/slp3/9.pdf
- https://www.youtube.com/watch?v=UNmqTiOnRfg
- https://www.youtube.com/watch?v=WCUNPb-5EYI
- https://www.youtube.com/watch?v=OuYtk9Ymut4

## <font color='#6629b2'>Dataset</font>

The Large Movie Review Datasetconsists of 50,000 movie reviews from [IMDB](http://www.imdb.com/). The ratings are on a 1-10 scale, but the dataset specifically contains "polarized" reviews: positive reviews with a rating of 7 or higher, and negative reviews with a rating of 4 or lower. There are an equal number of positive and negative reviews. 

You can download the dataset from: https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
! unzip imdb-dataset-of-50k-movie-reviews

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
 97% 25.0M/25.7M [00:02<00:00, 20.9MB/s]
100% 25.7M/25.7M [00:02<00:00, 12.4MB/s]
Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
'''Load the dataset into variable "reviews". You can truncate the dataset to keep a few hundred records if it's
    taking too long to process/train. Keep in mind, bigger the dataset, higher the accuracy score!
'''

reviews = pd.read_csv('/content/IMDB Dataset.csv', encoding = "ISO-8859-1")


reviews = reviews[:2000]

In [ ]:
#Split the data into train and split

from sklearn.model_selection import train_test_split

In [ ]:
# convert the above train test datasets into a pandas dataframe. You shoud have 4 dataframes.
# Name them train_reviews, test_reviews, train_sentiment, test_sentiment.
# In train_sentiment and test_sentiment, convert "positive" to 1 and "negative" to 0

train_reviews, test_reviews = train_test_split(reviews, test_size = 0.3)
train_sentiment, test_sentiment = train_test_split(reviews['sentiment'].replace(['positive', 'negative'], [1,0]), test_size = 0.3)


test_sentiment

24777    0
7173     1
20885    1
14250    1
2724     0
        ..
8752     1
24813    0
24978    0
15076    1
10414    1
Name: sentiment, Length: 7500, dtype: int64

In [ ]:
train_reviews.head()

,review,sentiment
19407,It's got action and fantasy mixed all together...,positive
21492,I don't know why this has gotten any decent re...,negative
13302,Almost the entire film takes place in a public...,positive
13047,Brides are dying at the altar and their corpse...,negative
21673,Wow-this one sucks. I'm gonna sum it up as qui...,negative


In [ ]:
train_reviews['review']

19407    It's got action and fantasy mixed all together...
21492    I don't know why this has gotten any decent re...
13302    Almost the entire film takes place in a public...
13047    Brides are dying at the altar and their corpse...
21673    Wow-this one sucks. I'm gonna sum it up as qui...
                               ...                        
16118    A 2006 online poll of Japan's top 100 favorite...
3073     Now this is the sort of film we used to get we...
4539     I just watched it last night and it was great....
13209    A very delightful bit of filmwork that should ...
14046    This is a film that every child should see bef...
Name: review, Length: 17500, dtype: object

## <font color='#6629b2'>Preparing the data</font>

###  <font color='#6629b2'>Tokenization</font>

The first preprocessing step is to tokenize each of the reviews into (lowercased) individual words, since the models will encode the reviews at the word level (rather than subword units like characters, for example). For this we'll use [spaCy](https://spacy.io/), which is a fast and extremely user-friendly library that performs various language processing tasks. Once you load a spaCy model for a particular language, you can provide any text as input to the model, e.g. encoder(text) and access its linguistic features.

In [ ]:
'''Lowercase and tokenise all the reviews in train_reviews using spacy'''

import spacy
encoder = spacy.load('en_core_web_sm')

def text_to_tokens(text_seqs):
    #complete this function that lowers and tokenizes the reviews
    doc = encoder(text_seqs)
    tokens = [token.text.lower() for token in doc]
    return tokens

train_reviews['Tokenized_Review'] = train_reviews.apply(lambda x: text_to_tokens(x['review']), axis=1)
#train_reviews['Tokenized_Review'] = text_to_tokens(train_reviews['review'])
train_reviews[['review','Tokenized_Review']][:10]

,review,Tokenized_Review
19407,It's got action and fantasy mixed all together...,"[it, 's, got, action, and, fantasy, mixed, all..."
21492,I don't know why this has gotten any decent re...,"[i, do, n't, know, why, this, has, gotten, any..."
13302,Almost the entire film takes place in a public...,"[almost, the, entire, film, takes, place, in, ..."
13047,Brides are dying at the altar and their corpse...,"[brides, are, dying, at, the, altar, and, thei..."
21673,Wow-this one sucks. I'm gonna sum it up as qui...,"[wow, -, this, one, sucks, ., i, 'm, gon, na, ..."
16405,This is the best thing Burt Reynolds ever did ...,"[this, is, the, best, thing, burt, reynolds, e..."
3513,One of the most unfairly maligned programmes o...,"[one, of, the, most, unfairly, maligned, progr..."
7858,Big spoiler right here: this film is B!A!D! Bu...,"[big, spoiler, right, here, :, this, film, is,..."
13148,I rented this film from Blockbust because of t...,"[i, rented, this, film, from, blockbust, becau..."
10299,I love the series! Many of the stereotypes por...,"[i, love, the, series, !, many, of, the, stere..."


###  <font color='#6629b2'>Lexicon</font>

Then we need to assemble a lexicon (aka vocabulary) of words that the model needs to know. Each tokenized word in the reviews is added to the lexicon, and then each word is mapped to a numerical index that can be read by the model. Since large datasets may contain a huge number of unique words, it's common to filter all words occurring less than a certain number of times, and replace them with some generic &lt;UNK&gt; token. The min_freq parameter in the function below defines this threshold. When assigning the indices, the number 1 will represent unknown words. The number 0 will represent "empty" word slots, which is explained below. Therefore "real" words will have indices of 2 or higher.

In [ ]:
'''Count tokens (words) in texts and add them to the lexicon'''

import pickle

def make_lexicon(token_seqs, min_freq=1, use_padding=False):
    # First, count how often each word appears in the text. Save this count in a dictionary called token_counts
    token_counts = {}

    # for row in train_reviews['Tokenized_Review']:
    #   for token in row:
    #     if token not in token_counts.keys():
    #       count = 0
    #       for r in train_reviews['Tokenized_Review']:
    #         for t in r:
    #           if token == t:
    #             count+=1
    #       token_counts[token] = count        

    train_tokens = train_reviews['Tokenized_Review'].explode('tokens')
    token_counts = train_tokens.value_counts().to_dict()
    

    # Then, assign each word to a numerical index, i.e save  all these words in a list. Filter words that occur less than or equal to min_freq times.
    
    lexicon = [token for token in token_counts if token_counts[token] <= min_freq]
    
    #create a dictionary lexicon that maps each word to its index. Note that indexes will start from 2,  index 0 is saved for padding and index 1 for unknown words ('<UNK>')
    lexicon = {lexicon[i]:i+2 for i in range(0,len(lexicon))}
    
    lexicon[u'<UNK>'] = 1 # Unknown words are those that occur fewer than min_freq times
    lexicon_size = len(lexicon)
    print(lexicon)

    print("LEXICON SAMPLE ({} total items):".format(lexicon_size))
    print(dict(list(lexicon.items())[:20]))
    
    return lexicon

lexicon = make_lexicon(token_seqs=train_reviews['Tokenized_Review'], min_freq=1)

{'boulevards': 2, '/>toss': 3, 'ranvijay': 4, 'bayridge': 5, 'eser': 6, 'calabrese': 7, '/>gã¶ta': 8, 'perspirer': 9, 'expediton': 10, 'deanna(erika': 11, 'beachwear': 12, 'corneal': 13, 'zohra': 14, 'anti-80s': 15, 'mahabharata': 16, 'd1': 17, 'reechi': 18, 'mollusk': 19, 'jockhood': 20, 'shabnam': 21, 'repugnantly': 22, 'gumbas': 23, 'deserves.i': 24, 'aspects-': 25, 'shoulders.<br': 26, 'eklavya': 27, 'gourmands': 28, 'cannibals.<br': 29, 'holstered': 30, '/>americans': 31, 'perfomances': 32, 'hustled': 33, 'boringest': 34, 'baby)creates': 35, 'baffle': 36, 'lolit': 37, 'developed.it': 38, '2003).<br': 39, 'revathi': 40, 'mistrust.<br': 41, 'wrong!<br': 42, 'treasurable': 43, 'huber': 44, '/>derivative': 45, 'fatherlands': 46, 'froggy': 47, 'aphorism': 48, 'aggh': 49, 'slimeball': 50, 'broke.<br': 51, 'ichabod': 52, 'cognac': 53, 'halfbacks': 54, 'cbgbomfug': 55, 'fitter': 56, 'inquired': 57, 'lillies': 58, 'preparating': 59, "ankhein','shola": 60, 'sovie': 61, 'soliders': 62, '/>ha

###  <font color='#6629b2'>From strings to numbers</font>

Once the lexicon is built, we can use it to transform each review from a list of string tokens into a list of numerical indices.

In [ ]:
'''Convert each review from a list of tokens to a list of numbers (indices)'''

def tokens_to_idxs(token_seqs, lexicon): 
    #complete this function to return a list of indexed tokens
    return [[lexicon[token] if token in lexicon.keys() else lexicon[u'<UNK>'] for token in row ] for row in token_seqs]
    

train_reviews['Review_Idxs'] = tokens_to_idxs(token_seqs=train_reviews['Tokenized_Review'], lexicon=lexicon)
                                   
train_reviews[['Tokenized_Review', 'Review_Idxs']][:10]

,Tokenized_Review,Review_Idxs
19407,"[it, 's, got, action, and, fantasy, mixed, all...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
21492,"[i, do, n't, know, why, this, has, gotten, any...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
13302,"[almost, the, entire, film, takes, place, in, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
13047,"[brides, are, dying, at, the, altar, and, thei...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
21673,"[wow, -, this, one, sucks, ., i, 'm, gon, na, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
16405,"[this, is, the, best, thing, burt, reynolds, e...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3513,"[one, of, the, most, unfairly, maligned, progr...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
7858,"[big, spoiler, right, here, :, this, film, is,...","[1, 1, 1, 1, 1, 1, 1, 1, 25841, 1, 1, 1, 1, 1,..."
13148,"[i, rented, this, film, from, blockbust, becau...","[1, 1, 1, 1, 1, 25839, 1, 1, 1, 1, 1, 1, 1, 1,..."
10299,"[i, love, the, series, !, many, of, the, stere...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 25826, 1, 1, 1,..."


In [ ]:
'''Encode reviews (train_reviews['Review_Idxs']) as bag-of-words vectors'''

def idx_seqs_to_bows(idx_seqs, matrix_length):
    #complete the function to return an array having bag-of-words vectors of the encoded reviews
    # hint: numpy.bincount()
    bow_seqs = np.bincount(idx_seqs)
    return bow_seqs
    
train_reviews.apply(lambda x: text_to_tokens(x['review']), axis=1)

bow_train_reviews = train_reviews.apply(lambda x: idx_seqs_to_bows(x['Review_Idxs'], 
                                     matrix_length=len(lexicon) + 1), axis=1).to_numpy() #add one to length for padding)

print("TRAIN INPUT:\n", bow_train_reviews)
print("SHAPE:", bow_train_reviews.shape, "\n")

#Showing an example mapping string words to counts
lexicon_lookup = {idx: lexicon_item for lexicon_item, idx in lexicon.items()}
lexicon_lookup[0] = ""
pd.DataFrame([(lexicon_lookup[idx], count) for idx, count in enumerate(bow_train_reviews[0])], 
                 columns=['Word', 'Count'])

TRAIN INPUT:
 [array([ 0, 67]) array([  0, 160,   0, ...,   0,   0,   1])
 array([  0, 204,   0, ...,   0,   0,   1]) ...
 array([  0, 432,   0, ...,   0,   0,   1]) array([ 0, 94])
 array([  0, 183,   0, ...,   0,   0,   1])]
SHAPE: (17500,) 



,Word,Count
0,,0
1,<UNK>,67


##  <font color='#6629b2'>Building a Recurrent Neural Network </font>



###  <font color='#6629b2'>Numerical lists to matrices</font>

The input representation for the RNN explicitly encodes the order of words in the review. We'll return to the lists of the word indices contained in train_reviews['Review_Idxs']. The input to the model will be these number sequences themselves. We need to put all the reviews in the training set into a single matrix, where each row is a review and each column is a word index in that sequence. This enables the model to process multiple sequences in parallel (batches) as opposed to one at a time. Using batches significantly speeds up training. However, each review has a different number of words, so we create a padded matrix equal to the length on the longest review in the training set. For all reviews with fewer words, we prepend the row with zeros representing an empty word position. This is why the number 0 was not assigned as a word index in the lexicon. We can tell Keras to ignore these zeros during training.

In [ ]:
import tensorflow

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def pad_idx_seqs(idx_seqs):
    
    #find the biggest review's length and save it in the variable below
    max_seq_len = max(len(row) for row in idx_seqs)
    return pad_sequences(idx_seqs,max_seq_len,padding='post')
    #pad all these indexed reviews and return these padded sequences
    #HINT: use pad_sequences function by keras
    


train_padded_idxs = pad_idx_seqs(train_reviews['Review_Idxs'])

print("TRAIN INPUT:\n", train_padded_idxs)
print("SHAPE:", train_padded_idxs.shape, "\n")

TRAIN INPUT:
 [[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]]
SHAPE: (17500, 2142) 



###  <font color='#6629b2'>Model Layers</font>
The RNN will have four layers:

**1. Input**: The input layer takes in the matrix of word indices.

**2. Embedding**: A [layer](https://keras.io/layers/embeddings/) that converts integer word indices into distributed vector representations (embeddings), which were introduced above. The difference here is that rather than plugging in embeddings from a pretrained model as before, the word embeddings will be learned inside the model itself. Thus, the input to the model will be the word indices rather than their embeddings, and the embedding values will change as the model is trained. The mask_zero=True parameter in this layer indicates that values of 0 in the matrix (the padding) will be ignored by the model.

**3. GRU**: A [recurrent (GRU) hidden layer](https://keras.io/layers/recurrent/), the central component of the model. As it observes each word in the review, it integrates the word embedding representation with what it's observed so far to compute a representation (hidden state) of the review at that timepoint. There are a few architectures for this layer - we use the GRU variation, Keras also provides LSTM or just the simple vanilla recurrent layer (see the materials at the bottom for an explanation of the difference). This layer outputs the last hidden state of the sequence (i.e. the hidden representation of the review after its last word is observed).

**4. Dense**: An output [layer](https://keras.io/layers/core/#dense) that predicts the rating for the review based on its GRU representation given by the previous layer. It has one dimension that contains a continuous value (the rating). Add a proper activation function.

###  <font color='#6629b2'>Parameters</font>

Our function for creating the RNN takes the following parameters:

**n_input_nodes**: As with the standard bag-of-words MLP, this is the number of unique words in the lexicon, plus one to account for the padding represented by 0 values. This indicates the number of rows in the embedding layer, where each row corresponds to a word.

**n_embedding_nodes**: the number of dimensions (units) in the embedding layer, which can be freely defined. Here, it is set to 300.

**n_hidden_nodes**: the number of dimensions in the GRU hidden layer. Like the embedding layer, this can be freely chosen. Here, it is set to 500.

In [ ]:
'''Create the model'''

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.layers import InputLayer

def create_rnn_model(n_input_nodes, n_embedding_nodes, n_hidden_nodes):
    
    model = tf.keras.Sequential()
    #complete this function to create a model and compile it having the 4 layers listed above.
    #Note: Layer 1 -  Technically the shape of this layer is (batch_size, len(train_padded_idxs)).
    # However, both the batch size and the length of the input matrix can be inferred from the input at training time. 
    # The batch size is implicitly included in the shape of the input, so it does not need to 
    # be specified as a dimension of the input. None can be given as placeholder for the input matrix length.
    # By defining it as None, the model is flexible in accepting inputs with different lengths.
    input_layer = Input(shape=(None,))

    model.add(input_layer)
    # Layer 2
    model.add(Embedding(input_dim=n_input_nodes, output_dim=n_embedding_nodes, mask_zero=True))
    # Layer 3
    model.add(tf.keras.layers.GRU(units=n_hidden_nodes))
    # Layer 4
    model.add(tf.keras.layers.Dense(units=1))
    model.add(tf.keras.layers.Activation("linear"))
    #Specify which layers are input and output, compile model with loss and optimization functions
    model.layers[0].trainable = False
    model.input_layers = [model.layers[0].input]
    model.output_layers = [model.layers[-1].output]

    model.compile(loss='mean_squared_error', optimizer='adam')
    return model


In [ ]:
rnn_model = create_rnn_model(n_input_nodes=len(lexicon) + 1, n_embedding_nodes=300, n_hidden_nodes=500)

###  <font color='#6629b2'>Training</font>

In [ ]:
'''
Train the rnn_model using the padded sequences and y=train_reviews['Rating'].
You need  to convert train_reviews['Rating'] to tensor before passing it as an argument
Hint: tf.convert_to_tensor
batch_size=20, epochs=5
'''

y = tf.convert_to_tensor(train_sentiment)
with tf.device('/device:GPU:0'):
  rnn_model.fit(x=train_padded_idxs,y=y, batch_size=20, epochs=5)


Epoch 1/5
875/875 [==============================] - 55s 61ms/step - loss: 0.2555
Epoch 2/5
875/875 [==============================] - 53s 60ms/step - loss: 0.2526
Epoch 3/5
875/875 [==============================] - 53s 60ms/step - loss: 0.2514
Epoch 4/5
875/875 [==============================] - 53s 61ms/step - loss: 0.2768
Epoch 5/5
875/875 [==============================] - 54s 62ms/step - loss: 0.2657


###  <font color='#6629b2'>Prediction</font>

In [ ]:
'''Put test reviews in padded matrix just how we did for train_reviews'''
test_reviews['Tokenized_Review'] = test_reviews.apply(lambda x: text_to_tokens(x['review']), axis=1)
test_reviews['Review_Idxs'] = tokens_to_idxs(token_seqs=test_reviews['Tokenized_Review'], lexicon=lexicon)

test_padded_idxs = pad_idx_seqs(test_reviews['Review_Idxs'])

print("TEST INPUT:\n", test_padded_idxs)
print("SHAPE:", test_padded_idxs.shape, "\n")

TEST INPUT:
 [[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]]
SHAPE: (7500, 1996) 



In [ ]:
'''Predict the ratings '''

#Since ratings are integers, need to round predicted rating to nearest integer
test_reviews['RNN_Pred_Rating'] = np.round(rnn_model.predict(test_padded_idxs)[:,0]).astype(int)

235/235 [==============================] - 8s 28ms/step


In [ ]:
test_reviews['RNN_Pred_Rating']

1878     0
15496    0
4252     0
5478     0
23284    0
        ..
20990    0
1426     0
2947     0
20008    0
6399     0
Name: RNN_Pred_Rating, Length: 7500, dtype: int64

###  <font color='#6629b2'>Evaluation</font>

In [ ]:
'''Evaluate the model with R^2'''

# print the r2 score
from sklearn.metrics import r2_score 
r2_score(y_true=tf.convert_to_tensor(test_sentiment),y_pred=test_reviews['RNN_Pred_Rating'])

-1.0080365551899115

On the full test dataset of 25,000 reviews, the $R^2$ for this model is 0.622525. So the RNN outperforms the continuous bag-of-words MLP as well as the standard bag-of-words approach.
Your score might not be good because we're training on only 100-200 reviews.

### <font color='#6629b2'>Visualizing data inside the model</font>

To help visualize the data representation inside the model, we can look at the output of each layer in a model individually. Keras' Functional API lets you derive a new model with the layers from an existing model, so you can define the output to be a layer below the output layer in the original model. Calling predict() on this new model will produce the output of that layer for a given input. Of course, glancing at the numbers by themselves doesn't provide any interpretation of what the model has learned (although there are opportunities to [interpret these values](https://medium.com/civis-analytics/interpreting-and-visualizing-neural-networks-for-text-processing-e9dff0da9c22), but seeing them verifies the model is just a series of transformations from one matrix to another. The model stores its layers as the list model.layers, and you can retrieve specific layer by its position index in the model.

In [ ]:
'''Showing the output of the RNN embedding layer (second layer) for the test reviews'''

embedding_layer = Model(inputs=rnn_model.layers[0].input, 
                        outputs=rnn_model.layers[1].output) #embedding layer is 2nd layer (index 1)
embedding_output = embedding_layer.predict(test_padded_idxs)
print("EMBEDDING LAYER OUTPUT SHAPE:", embedding_output.shape)
print(embedding_output[0])

235/235 [==============================] - 7s 28ms/step
EMBEDDING LAYER OUTPUT SHAPE: (7500, 500)
[ 0.20628121  0.9997184   0.9990749   0.39019015  0.99902624  0.07002313
 -0.98269427 -0.98030645 -0.59855765 -0.9989881   0.16166073  0.434796
  0.2284604   0.2569677  -0.98094684 -0.9518246   0.36068788  0.49846935
 -0.18406367 -0.33842704  0.9910256  -0.9803366   0.14072998  0.9667437
  0.9998365   0.9970272   0.9989173   0.14030662  0.19445096  0.17207636
 -0.9391894   0.058645    0.09532781 -0.15889454 -0.12531932 -0.08614982
  0.48183617 -0.99089056  0.14119035  0.9997313   0.5337059   0.9860413
  0.10096823  0.14879473  0.9917945   0.92266566 -0.8400336   0.10751868
  0.5445563   0.55371225 -0.03153863  0.36167654  0.18689522  0.95708364
  0.9993666  -0.893788   -0.99978733  0.19907428 -0.99717534  0.56707877
 -0.9996105   0.99889684 -0.9799792   0.99441165 -0.9783073  -0.9798807
  0.8238285   0.9732176  -0.9988569   0.69126135  0.97515124 -0.9750932
 -0.12463682 -0.10152131  0.9865

## <font color='#6629b2'>Conclusion</font>

As mentioned above, the models shown here could be applied to any task where the goal is to predict a score for a particular sequence. For ratings prediction, this output is ordinal, but it could also be categorical with a few simple changes to the output layer of the model. 

## <font color='#6629b2'>More resources</font>

Yoav Goldberg's book [Neural Network Methods for Natural Language Processing](http://www.morganclaypool.com/doi/abs/10.2200/S00762ED1V01Y201703HLT037) is a thorough introduction to neural networks for NLP tasks in general.

If you'd like to learn more about what Keras is doing under the hood, there is a [Theano tutorial](http://deeplearning.net/tutorial/lstm.html) that also applies an RNN to sentiment prediction, using the same dataset here

Andrej Karpathy's blog post [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) is very helpful for understanding the mathematical details of an RNN, applied to the task of language modeling. It also provides raw Python code with an implementation of the backpropagation algorithm.

TensorFlow also has an RNN language model [tutorial](https://www.tensorflow.org/versions/r0.12/tutorials/recurrent/index.html) using the Penn Treebank dataset

Chris Olah provides a good [explanation](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) of how LSTM RNNs work (this explanation also applies to the GRU model used here)

Denny Britz's [tutorial](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) documents well both the technical details of RNNs and their implementation in Python.